In [2]:
# Import the functions used in this project
import awesome_functions as af
import decode_utils as du

# Pretty display for notebooks
%matplotlib inline

# Ignore the warnings
import warnings
warnings.filterwarnings('ignore')

# Load the dataset
train = pd.read_csv("../asset/train.csv")
test = pd.read_csv("../asset/test.csv")
submission = pd.read_csv("../asset/sample_submission.csv")

# Success - Display the first record
print("Train data : ", train.shape)
print("Test  data : ", test.shape)
print("submission  data : ", submission.shape)

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import pickle as pkl
slack_url = pkl.load(open("Slack_url/send_url.pickle", "rb"))

import xgboost
from sklearn.metrics import log_loss
from sklearn.naive_bayes import MultinomialNB
from scipy.sparse import csr_matrix
def fitNaiveBayesModel(X, y):
    return MultinomialNB(alpha=0.0).fit(X, y)

# 원본을 유지하기 위해서 카피
df_train = train.copy()
df_test = test.copy()
df_submission = submission.copy()

df_train_dd = pd.read_csv("Feature_matrix/df_train_dd_201807291831.csv")
df_test_dd = pd.read_csv("Feature_matrix/df_test_dd_201807291855.csv")
# df_train_fl = pd.read_csv("Feature_matrix/df_train_fl_201807291845.csv")
# df_test_fl = pd.read_csv("Feature_matrix/df_test_fl_201807291908.csv.csv")
# df_ratio = pd.read_csv("Feature_matrix/df_new_has_all_the_ratio_201808031807.csv")

Train data :  (647054, 7)
Test  data :  (653646, 6)
submission  data :  (95674, 39)


In [9]:
def get_pivor_df(df, col):
    return pd.pivot_table(data= df, index="VisitNumber", fill_value=0,\
                          values="ScanCount", columns=col, aggfunc=np.sum)

def get_rid_of_zero_units(df):
    return np.where(df < 0, 0, df)

In [10]:
def getSpecifiedVisitNumberData(df_train, vn):
    display(df_train[df_train.VisitNumber == vn])
#     display(df_train_dd[df_train_dd.VisitNumber == vn])

In [8]:
def get_real_category_cnt(df_, col = "DepartmentDescription", thresould=1.0):
    df = df_.copy()
    df_count = df.groupby(["VisitNumber", col]).sum()["ScanCount"].reset_index(name="Sc_sum")
    
#     display(df_count)
    # sc_count가 음수인 아이템은 같은 날 산 아이템이라 볼 수 없다.
    # 리턴하기위해 가져온 것들이므로 월마트를 나갈 때 들고가는 아이템이라 보지않아도된다. 
    # 그런 아이템은 모두 nan으로 바꾸고 drop한다.
    df_count["Sc_sum"] = np.where(df_count["Sc_sum"] < 0, np.nan, df_count["Sc_sum"])
    df_count = df_count.dropna()
#     display(df_count)
    
    # 남은 아이템들만 더해주면 위에서 지정해준 컬럼의 유니크한 개수만 남는다.
    df_count = df_count.groupby("VisitNumber").sum()["Sc_sum"].reset_index(name="Sc_sum")
#     display(df_count)
    
    return list(df_count[df_count.Sc_sum == thresould].VisitNumber.unique())

In [20]:
# import decode_utils as du
# import awesome_functions as af
from sklearn.metrics import confusion_matrix
from IPython.display import display, Markdown

def getAccuracy(y_true, y_pred, data_length, i):
    """
        y_true : 원래 타겟 컬럼의 데이터를 넣어준다.
        y_pred : 예측한 값을 넣어준다.
        data_length : 예측한 데이터의 총 개수를 넣어준다.
    """
    accuracy = round(np.trace(confusion_matrix(y_true, y_pred))/data_length, 4)
    display(Markdown("##### Accuracy : " + str(accuracy) + ", smoothing : " + str(i)))
    return accuracy

def fitNaiveBayesModel_smoothing(X, y, a = 1.0):
    return MultinomialNB(alpha=a).fit(X, y)

def __combine_dd_fl(df_dd, df_fl):
    col_we_dont_need = ["VisitNumber", "TripType", 'Return', 'Monday',\
                        'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    cols_we_need = [col for col in df_dd.columns if col not in col_we_dont_need]
    cols = [col for col in cols_we_need if col != "VisitNumber"]
    return du.concatDf(df_dd[cols], df_fl)

def get_df_has_specified_vn_list(vn_list, df=None, df_dd=None, df_fl=None, is_in=True,
                                 is_need_to_make_df = False):
    if is_need_to_make_df:
        df = __combine_dd_fl(df_dd, df_fl)
        print("Concat 완료.")
    if is_in:
        return df[df.VisitNumber.isin(vn_list)]
    return df[~df.VisitNumber.isin(vn_list)]

def get_df_dd_has_specified_vn_list(df_, vn_list, is_in=True):
    df = df_.copy()
    dd_cols = df.columns
    if ("MENS WEAR" in dd_cols) and ("MENSWEAR" in dd_cols):
        df["MENS WEAR"] = df["MENSWEAR"] + df["MENS WEAR"]
        df.drop("MENSWEAR", axis=1, inplace = True)
    if is_in:
        return df[df.VisitNumber.isin(vn_list)]
    return df[~df.VisitNumber.isin(vn_list)]

def test_multinomial(df, early_stop = 5):
    X, y = af.get_df_to_fit(df)
    a = np.arange(0, 0.16, 0.01)
    accuracy_ = 0
    count = 0
    result = []
    alpha = []
    for i in a:
        if count >= early_stop:
            best_score = np.array(result).max()
            idx = result.index(best_score)
            display(Markdown("## Best_score : " + str(best_score) + ", Alpha : " + str(alpha[idx])))
            return
        model_nb = fitNaiveBayesModel_smoothing(X, y, i)
        y_pred = model_nb.predict(X)
        accuracy = getAccuracy(y, y_pred, len(X), i)
        if accuracy_ > accuracy:
            count += 1
        accuracy_ = accuracy
        result.append(accuracy)
        alpha.append(i)
    best_score = np.array(result).max()
    idx = result.index(best_score)
    display(Markdown("## Best_score : " + str(best_score) + ", Alpha : " + str(alpha[idx])))
#     return best_score, alpha[idx]

In [6]:
df_decoded = du.decodeStuffNeedsToBeDecoded(df_train)

In [11]:
vn_dd_one = get_real_category_cnt(df_decoded)

In [12]:
vn_fl_one = get_real_category_cnt(df_decoded, "FinelineNumber")

In [21]:
df_to_test_dd_only = get_df_dd_has_specified_vn_list(df_train_dd, vn_dd_one)

In [22]:
test_multinomial(df_to_test_dd_only)

##### Accuracy : 0.8086, smoothing : 0.0

##### Accuracy : 0.8086, smoothing : 0.01

##### Accuracy : 0.8086, smoothing : 0.02

##### Accuracy : 0.8086, smoothing : 0.03

##### Accuracy : 0.8085, smoothing : 0.04

...

##### Accuracy : 0.8084, smoothing : 0.12

##### Accuracy : 0.8084, smoothing : 0.13

##### Accuracy : 0.8084, smoothing : 0.14

##### Accuracy : 0.8086, smoothing : 0.15

## Best_score : 0.8086, Alpha : 0.0

In [28]:
# df_to_test = get_df_has_specified_vn_list(vn_fl_one, df_dd=df_train_dd, df_fl=df_train_fl, is_need_to_make_df=True)

Concat 완료.


In [34]:
# test_multinomial(df_to_test)

##### Accuracy : 0.9331, smoothing : 0.0

##### Accuracy : 0.9372, smoothing : 0.01

##### Accuracy : 0.9387, smoothing : 0.02

##### Accuracy : 0.9383, smoothing : 0.03

##### Accuracy : 0.9385, smoothing : 0.04

##### Accuracy : 0.9374, smoothing : 0.05

##### Accuracy : 0.9365, smoothing : 0.06

##### Accuracy : 0.9355, smoothing : 0.07

##### Accuracy : 0.9351, smoothing : 0.08

## Best_score : 0.9387, Alpha : 0.02

In [35]:
# df_to_test = get_df_has_specified_vn_list(vn_dd_one, df_dd=df_train_dd, df_fl=df_train_fl, is_need_to_make_df=True)

Concat 완료.


In [36]:
# test_multinomial(df_to_test)

##### Accuracy : 0.9336, smoothing : 0.0

##### Accuracy : 0.9377, smoothing : 0.01

##### Accuracy : 0.9395, smoothing : 0.02

##### Accuracy : 0.939, smoothing : 0.03

##### Accuracy : 0.9388, smoothing : 0.04

##### Accuracy : 0.9377, smoothing : 0.05

##### Accuracy : 0.9367, smoothing : 0.06

##### Accuracy : 0.9357, smoothing : 0.07

## Best_score : 0.9395, Alpha : 0.02